In [1]:
!pip install ctransformers huggingface-hub -q

In [2]:
import os
from ctransformers import AutoModelForCausalLM
from huggingface_hub import hf_hub_download

# Définition du modèle à utiliser
# On choisit un modèle Mistral 7B Instruct, quantifié en 4-bit (Q4_K_M)
# C'est un bon équilibre performance / taille pour une utilisation CPU
model_repo = "TheBloke/Mistral-7B-Instruct-v0.1-GGUF"
model_file = "mistral-7b-instruct-v0.1.Q4_K_M.gguf"

# Télécharger le modèle (si pas déjà présent)
print(f"Téléchargement du modèle: {model_file}...")
model_path = hf_hub_download(repo_id=model_repo, filename=model_file)
print(f"Modèle téléchargé dans: {model_path}")

/home/matheo/.conda/envs/projet_ia5/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Téléchargement du modèle: mistral-7b-instruct-v0.1.Q4_K_M.gguf...
Modèle téléchargé dans: /home/matheo/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.1-GGUF/snapshots/731a9fc8f06f5f5e2db8a0cf9d256197eb6e05d1/mistral-7b-instruct-v0.1.Q4_K_M.gguf


In [3]:
# Configuration et chargement du modèle
print("Chargement du modèle sur le CPU (cela peut prendre un moment)...")

# Configuration pour CTransformers
# gpu_layers=0 signifie qu'aucune couche n'est déchargée sur le GPU (utilisation CPU pure)
llm = AutoModelForCausalLM.from_pretrained(
    model_path,
    model_type="mistral",
    context_length=4096,  # Longueur de contexte max (texte + réponse)
    gpu_layers=0          # <-- TRÈS IMPORTANT: Force l'utilisation du CPU
)

print("Modèle chargé avec succès.")

Chargement du modèle sur le CPU (cela peut prendre un moment)...
Modèle chargé avec succès.


In [4]:
def traduire_style_epoque(texte_original: str, epoque_originale: str, epoque_cible: str) -> str:
    """
    Utilise le LLM pour réécrire un texte dans le style d'une autre époque.
    """
    
    # Un "prompt" système pour guider le modèle.
    # Le format [INST]...[/INST] est spécifique à Mistral Instruct.
    prompt = f"""[INST] Tu es un expert en histoire de la littérature française et un écrivain.
Ta tâche est de réécrire le texte suivant, qui a été écrit dans le style de l'époque "{epoque_originale}", pour qu'il corresponde parfaitement au style (vocabulaire, grammaire, syntaxe) de l'époque "{epoque_cible}".
Le sens, les idées et le contenu du texte original doivent être préservés au maximum. Ne fais que modifier le style.

TEXTE ORIGINAL ({epoque_originale}):
{texte_original}

RÉÉCRITURE ({epoque_cible}):
[/INST]"""

    print(f"Génération de la traduction stylistique de '{epoque_originale}' vers '{epoque_cible}'...")
    print("--------------------------------------------------")
    
    # Appel du modèle
    # 'temperature' contrôle la créativité (0.7 est un bon équilibre)
    # 'max_new_tokens' est la longueur max de la réponse. Augmentez si vos textes sont longs.
    # ATTENTION: La génération sur CPU sera LENTE.
    reponse = llm(
        prompt, 
        temperature=0.7, 
        max_new_tokens=1024, 
        top_k=40
    )
    
    return reponse

In [5]:
# --- Exemple 1 : Moderne vers Ancien ---

texte_moderne = """
Salut les gars,
J'ai checké le nouveau projet sur le drive, c'est vraiment un truc de ouf.
Mon boss m'a dit que le client est super content et que le lancement est prévu pour ce week-end. 
On va pouvoir décompresser, c'est trop cool.
"""
epoque_source_1 = "2024"
epoque_cible_1 = "1670 (style Molière ou La Fontaine)"

# Lancement de la génération
texte_traduit_1 = traduire_style_epoque(texte_moderne, epoque_source_1, epoque_cible_1)

# Affichage
print("\n--- TEXTE ORIGINAL (" + epoque_source_1 + ") ---")
print(texte_moderne)
print("\n--- TEXTE TRADUIT (" + epoque_cible_1 + ") ---")
print(texte_traduit_1)

Génération de la traduction stylistique de '2024' vers '1670 (style Molière ou La Fontaine)'...
--------------------------------------------------

--- TEXTE ORIGINAL (2024) ---

Salut les gars,
J'ai checké le nouveau projet sur le drive, c'est vraiment un truc de ouf.
Mon boss m'a dit que le client est super content et que le lancement est prévu pour ce week-end. 
On va pouvoir décompresser, c'est trop cool.


--- TEXTE TRADUIT (1670 (style Molière ou La Fontaine)) ---
 Salut mes amis,
J'ai vérifié le dernier projet sur mon bureau à la nuit, et je suis très content que cela soit bien arrivé.
Mon patron m'a informé que le client est extrêmement satisfait et que le lancement est prévu pour ce weekend. 
On peut décomprimer, c'est trop magnifique.


In [6]:
# --- Exemple 2 : Ancien vers Moderne ---

texte_ancien = """
Monseigneur,
Je me fais l'honneur de vous adresser la présente missive afin de requérir votre bienveillance. 
Les affaires de votre domaine prospèrent, mais la météo capricieuse menace les récoltes.
Je crains fort que la disette ne frappe nos manants si le ciel ne nous est point plus clément.
Votre très humble et dévoué serviteur.
"""
epoque_source_2 = "1720"
epoque_cible_2 = "2024 (style email professionnel)"

# Lancement de la génération
texte_traduit_2 = traduire_style_epoque(texte_ancien, epoque_source_2, epoque_cible_2)

# Affichage
print("\n--- TEXTE ORIGINAL (" + epoque_source_2 + ") ---")
print(texte_ancien)
print("\n--- TEXTE TRADUIT (" + epoque_cible_2 + ") ---")
print(texte_traduit_2)

Génération de la traduction stylistique de '1720' vers '2024 (style email professionnel)'...
--------------------------------------------------

--- TEXTE ORIGINAL (1720) ---

Monseigneur,
Je me fais l'honneur de vous adresser la présente missive afin de requérir votre bienveillance. 
Les affaires de votre domaine prospèrent, mais la météo capricieuse menace les récoltes.
Je crains fort que la disette ne frappe nos manants si le ciel ne nous est point plus clément.
Votre très humble et dévoué serviteur.


--- TEXTE TRADUIT (2024 (style email professionnel)) ---
 Subject: Inquiry of Goodwill - Impact of Unfavorable Weather on Crop Yield

Dear Sir,

I trust this message finds you well and in good spirits. I am reaching out to kindly request your assistance with a matter of utmost concern for our community.

As you are aware, the agricultural sector has been thriving under your leadership. However, recent weather conditions have posed an unprecedented challenge - excessive rainfall follow